In [1]:
import sys
sys.path.append('C:\\Users\\dg\\Documents\\python\\diffusion-net\\experiments\\regression_0')

In [9]:
from matplotlib.patches import Rectangle

In [2]:
from scipy.stats import pearsonr
from ga_population import *

paths = {
    51: Path(r"D:\resynth\run_51_52\run00051_resynth\2025-09-16-15-34-44\opts_and_metadata.pt"),
    42: Path(r"D:\resynth\run_42_43\run00042_resynth\2025-09-11-16-14-13\opts_and_metadata.pt"),
    48: Path(r"D:\resynth\run_48_49\run00048_resynth\2025-09-12-13-05-14\opts_and_metadata.pt"),
    38: Path(r"D:\resynth\run_38_39\run00038_resynth\2025-09-13-10-59-36\opts_and_metadata.pt"),
    20: Path(r"D:\resynth\run_20_21\run00020_resynth\2025-09-14-15-06-07\opts_and_metadata.pt"),
     9: Path(r"D:\resynth\run_09_10\run00009_resynth\2025-09-15-10-17-13\opts_and_metadata.pt"),
}
readers = {run_id: Readers.from_file(f) for run_id, f in paths.items()}

def get_readers(visible_mode: UseVisibleMode | None):
    out = dict()
    tgt = str(visible_mode)
    for run_id, rs in readers.items():
        idx = next(i for i, v in rs.hparams.use_visible.items() if str(v) == tgt)
        out[run_id] = rs[idx]
    return out

def get_src_models(visible_mode: UseVisibleMode | None):
    readers = get_readers(visible_mode)
    return {
        run_id: SourceModel.from_reader(reader, cp_data_file=Path(SourceModels.cp_data_files(run_id)))
        for run_id, reader in readers.items()
    }

# _sms = get_src_models(UseVisibleMode(False, False))
_sms = get_src_models(None)
sms = SourceModels(list(_sms.values()))

pms = [p.cached() for p in ProbeMeshSpec.defined()]

recache = False
cp_pm_preds = sms.corpus_probe_mesh_preds(pms=pms, cache_file='cp_pm_preds.hdf', recache=recache)
cp_responses = sms.corpus_observed_responses(cp_pm_preds=cp_pm_preds, cache_file='cpm_responses.hdf', recache=recache)

# vertex weights is a n_stim list of VertexWeights, wehere VertexWeights.weights is (n_vertices, n_channels_total)
# cp_vertex_weights = sms.get_weights(pms, expt_kwargs=dict(outputs_at='vertices'))
# torch.save(cp_vertex_weights, 'cp_vertex_weights.pt')
cp_vertex_weights = torch.load('cp_vertex_weights.pt')

drop = [
    'Barcelona Door Handle Brass_wip_v3_orig.png',
    'JF0N2N0A_Chipmunk_FindFood_v4_orig_Main.png',
    'door_knob_v3_orig.png',
    'Door Handle 015 v3_orig.png',
    'Muuto_Tip_Table_Light_v3_orig.png',
    'FLOOR_LAMP_02_v3_orig.png',
    'FLOOR_LAMP_03_v3_orig.png',
    'liquor_bottles_v4_orig_Ballantine.png',
    'liquor_bottles_v4_orig_Bombay.png',
    'Vintage_Ring_Shaped_Door_Knocker_v3_orig.png',
    'Lamp_v3_orig.png',
    'Coffee_Machine_v3_orig.png',
    'Lemon_v3_orig.png',
    'Mouse with Mousepad_v3_orig.png',
]
keep = [i for i, pm in enumerate(pms) if pm.index not in drop]
pms = np.array(pms)[keep].tolist()
cp_vertex_weights = np.array(cp_vertex_weights)[keep].tolist()
cp_responses_vr = cp_responses.iloc[keep]

C:\Users\dg\.conda\envs\diffnet4\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [12]:
ax = plt.gca()

In [ ]:
ax.vi

In [51]:
from tqdm.notebook import tqdm

def _scatter(ax, obs, preds, ttl):
    ax.plot(obs, preds, 'k.')
    ax.plot(obs, preds, 'k.')
    r = pearsonr(obs, preds).statistic
    ax.set_title(f'{ttl} (r={r:.2f})')
    ax.set_xlabel('Observed response')
    ax.set_ylabel('Predicted responses')
    ax.axis('square')


def scatter_plots(reader, channel, global_ch_idx, fig=None, ttl=''):
    if fig:
        axs = fig.subplots(1, 3)
    else:
        fig, axs = plt.subplots(1, 3, figsize=(20, 5))

    sd = reader.scatter_data

    axs[0].axis('off')
    if ttl:
        axs[0].set_title(ttl)
    
    _scatter(axs[1], *sd.loc(scene_ids=reader.train_scenes, channel=channel), ttl='Train GA')
    _scatter(axs[2], *sd.loc(scene_ids=reader.test_scenes, channel=channel), ttl='Test GA')

    return fig


def plot_single_channel_vertex_weights(
        pms: list[ProbeMesh],
        vertex_weights: list[VertexWeights],
        channel_idx: int,
        img_sz=(5, 5),
        grid_sz=None,
        fig=None,
        weights_clim_pctile=(1, 99),
):
    n_stim = len(pms)
    assert len(vertex_weights) == n_stim
    if grid_sz is None:
        _n = int(np.ceil(np.sqrt(n_stim)))
        grid_sz = (_n, _n)

    nr, nc = grid_sz
    img_sz = np.array(img_sz)

    if fig:
        axs = fig.subplots(nr, nc)
    else:
        fig, axs = plt.subplots(nr, nc, figsize=(img_sz[0] * nc, img_sz[1] * nr), squeeze=False)

    axs = axs.reshape(-1)

    for pm, vw, ax in zip(pms, vertex_weights, axs):
        rendered_weights = pm.render(
            weights=vw.weights[:, channel_idx],
            ground=False,
            show_scalar_bar=False,
            window_size=(1024, 1024),
            weights_clim_pctile=weights_clim_pctile,
        )
        ax.imshow(rendered_weights)
        ax.axis('off')

    for ax in axs[n_stim:]:
        ax.axis('off')

    return fig

def plot_single_channel_renders(
        pms: list[ProbeMesh],
        channel_idx: int,
        img_sz=(5, 5),
        grid_sz=None,
        fig=None,
        responses=None,
        preds=None,
):
    n_stim = len(pms)
    if grid_sz is None:
        _n = int(np.ceil(np.sqrt(n_stim)))
        grid_sz = (_n, _n)

    nr, nc = grid_sz
    img_sz = np.array(img_sz)

    if fig:
        axs = fig.subplots(nr, nc)
    else:
        fig, axs = plt.subplots(nr, nc, figsize=(img_sz[0] * nc, img_sz[1] * nr), squeeze=False)

    axs = axs.reshape(-1)

    if responses is None:
        responses = [None for _ in pms]
    if preds is None:
        preds = [None for _ in pms]

    bar_w = 0.025
    for pm, ax, r, p in zip(pms, axs, responses, preds):
        ax.imshow(pm.img, extent=(0, 1, 0, -1))
        if r is not None:
            ax.add_patch(Rectangle((0, 0), bar_w, -r, facecolor='red'))
        if p is not None:
            ax.add_patch(Rectangle((bar_w, 0,), bar_w, -p, facecolor='blue'))
                         
        ax.axis('off')
        

    for ax in axs[n_stim:]:
        ax.axis('off')

    return fig

def load_unit_best(model, channel_idx: int, n_best: int):
    meta = model.reader.metadata
    scenes, responses, weights, fit_fns = meta.load_data(weights=meta.weight_error)
    channel = meta.channel[channel_idx]
    unit_responses = responses.iloc[:, channel_idx].sort_values(ascending=False)
    unit_best_scenes = unit_responses.index[:n_best]

    # Load the meshes
    ga_pms = list(ProbeMesh.load_ga_stim(meta.opts.data_dir, scenes.loc[unit_best_scenes]))

    # Present the meshes to the network to get vertex weights
    dataset = GaDataset(
        df=scenes.loc[unit_best_scenes],  # type: ignore
        responses=responses.loc[unit_best_scenes],  # type: ignore
        root_dir=meta.opts.data_file.parent,
        k_eig=meta.k_eig,
        op_cache_dir=meta.opts.data_dir / 'op_cache',
        file_mode=meta.opts.mesh_file_mode,
        weights=meta.weight_error,
        use_visible=meta.use_visible,
        use_color=meta.use_color,
        norm_verts=meta.norm_verts,
        features=meta.input_features,
        augment=None,
    )
    dataloader = DataLoader(dataset, shuffle=False, batch_size=None)
    expt = model.reader.experiment(outputs_at='global_mean')
    _, ga_preds = expt.predict(dataloader, agg_fn=np.stack)
    ga_preds = ga_preds[:, channel_idx]
    
    expt.model.outputs_at = 'vertices'
    _ga_obs, ga_weights = expt.predict(dataloader, agg_fn=lambda x: x)
    ga_weights = [VertexWeights(weights=w) for w in ga_weights]

    return ga_pms, ga_weights, unit_responses.values[:n_best], ga_preds

# def load_unit_best(model, channel_idx: int, n_best: int):
#     meta = model.reader.metadata
#     scenes, responses, weights, fit_fns = meta.load_data(weights=meta.weight_error)
#     channel = meta.channel[channel_idx]

#     n_sample = n_best * 4
#     unit_responses = responses.iloc[:, channel_idx].sort_values(ascending=False)
#     ga_resps = unit_responses.values[:n_sample]
#     unit_best_scenes = unit_responses.index[:n_sample]

#     # Load the meshes
#     ga_pms = list(ProbeMesh.load_ga_stim(meta.opts.data_dir, scenes.loc[unit_best_scenes]))

#     # Present the meshes to the network to get vertex weights
#     dataset = GaDataset(
#         df=scenes.loc[unit_best_scenes],  # type: ignore
#         responses=responses.loc[unit_best_scenes],  # type: ignore
#         root_dir=meta.opts.data_file.parent,
#         k_eig=meta.k_eig,
#         op_cache_dir=meta.opts.data_dir / 'op_cache',
#         file_mode=meta.opts.mesh_file_mode,
#         weights=meta.weight_error,
#         use_visible=meta.use_visible,
#         use_color=meta.use_color,
#         norm_verts=meta.norm_verts,
#         features=meta.input_features,
#         augment=None,
#     )
#     dataloader = DataLoader(dataset, shuffle=False, batch_size=None)
#     expt = model.reader.experiment(outputs_at='global_mean')
#     _, ga_preds = expt.predict(dataloader, agg_fn=np.stack)
#     ga_preds = ga_preds[:, channel_idx]
    
#     expt.model.outputs_at = 'vertices'
#     _ga_obs, ga_weights = expt.predict(dataloader, agg_fn=lambda x: x)
#     ga_weights = [VertexWeights(weights=w) for w in ga_weights]

#     objective = ga_resps - np.abs(ga_resps - ga_preds)
#     idx = np.argsort(objective)[-n_best:][::-1]

#     ga_pms = [ga_pms[i] for i in idx]
#     ga_weights = [ga_weights[i] for i in idx]
#     ga_resps = ga_resps[idx]
#     ga_preds = ga_preds[idx]

#     return ga_pms, ga_weights, ga_resps, ga_preds

In [44]:
ok_channels = {
    9: [0, 2, 23, 31],
    20: [3, 30, 31],
    38: [13, 14, 17, 18, 29, 31],
    42: [],
    48: [11, 13, 20],
    51: [0, 14, 31],
}

ok_channels = {
    9: [],
    20: [],
    38: [14],
    42: [],
    48: [20],
    51: [0],
}

In [52]:
plt.ioff()
plt.rcParams['font.size'] = 16

root_folder = Path(r'C:\Users\dg\Documents\python\diffusion-net\experiments\regression_0\poster2025\exported_channels')

n_best = 5
n_cp = len(pms)
cp_grid_nr = int(np.ceil(n_cp / n_best))
cp_grid_sz = (cp_grid_nr, n_best)


for global_ch_idx, (model_idx, model_ch_idx) in enumerate(tqdm(cp_responses.columns, total=cp_responses.shape[1])):
    model = sms[model_idx]
    reader = model.reader
    channel = reader.metadata.channel[model_ch_idx]
    
    expt_name = reader.trained_file.parts[2]
    _, ga_run_id, cp_run_id = expt_name.split('_')
    if channel not in ok_channels[int(ga_run_id)]:
        continue
    
    ttl =f'{expt_name} - ch{channel}'
    fig = plt.figure(figsize=(14, 14), constrained_layout=True)
    # fig.suptitle(ttl, fontsize=24)
    subfigs = fig.subfigures(5, 1, height_ratios=[2, 1, 1, 1, 1], hspace=.05)

    scatter_plots(reader, channel, global_ch_idx, fig=subfigs[0], ttl=ttl)
   
    ga_pms, ga_vertex_weights, ga_unit_responses, ga_unit_preds = load_unit_best(model, model_ch_idx, n_best=n_best)
    plot_single_channel_renders(ga_pms, channel_idx=model_ch_idx, grid_sz=(1, n_best), fig=subfigs[1], responses=ga_unit_responses, preds=ga_unit_preds)
    plot_single_channel_vertex_weights(ga_pms, ga_vertex_weights, channel_idx=model_ch_idx, grid_sz=(1, n_best), fig=subfigs[2])

    cp_unit_responses = cp_responses_vr.loc[:, (model_idx, model_ch_idx)].reset_index(drop=True).sort_values(ascending=False).head(n_best)
    cp_idxs = cp_unit_responses.index.values
    cp_pms, cp_vws = [pms[i] for i in cp_idxs], [cp_vertex_weights[i] for i in cp_idxs]
    cp_unit_preds = cp_pm_preds.loc[[pm.index for pm in cp_pms], (model_idx, model_ch_idx)]
    
    plot_single_channel_renders(cp_pms, channel_idx=model_ch_idx, grid_sz=(1, n_best), fig=subfigs[3], responses=cp_unit_responses, preds=cp_unit_preds)
    plot_single_channel_vertex_weights(cp_pms, cp_vws, channel_idx=model_ch_idx, grid_sz=(1, n_best), fig=subfigs[4])

    fig.set_linewidth(10)
    fig.savefig(root_folder / (ttl + '.png'))
    plt.close(fig)
    # break

  0%|          | 0/86 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [38]:
plt.rcParams['font.size']

10.0